In [1]:
from pprint import pprint

This illustrates the process of deploying an SQLite database to handle the SSM data in the ICGC Data Releases. 

For this, we'll use the next schema:

![schema](database_schema.png)

For this purpose, and to facilitate the handling of the data with Python, we'll be using the Object Relational Mapper library called [PonyORM](https://ponyorm.com/). This library helps map Python classes to SQL tables, to handle records as Python objects and to make queries as if making Python generators.

In [2]:
def head(iterator, n):
    'Return the first n items of the given iterator.'
    for i, item in enumerate(iterator):
        if i>=n:
            break
        yield item

In [3]:
from collections import namedtuple, OrderedDict
# The types of the fields
# The format is that of the datashape module

# Main fields defining a mutation
main_fields = [('chromosome', 'string'),
               ('GRCh37_pos', 'int32'),
               ('mutation_id', 'string'), 
               ('reference_allele', 'string'), 
               ('mutated_allele', 'string'),
               ('quality', 'string'), 
               ('filter', 'string')]

# Fields which may have several values
tuple_fields = [('consequence','string'), 
                ('occurrence_by_project','string')]

# Fields specifying occurrence over all projects
occurrence_global_fields = [('affected_donors', 'int32'),
                            ('mutation', 'string'),
                            ('project_count','int32'), 
                            ('tested_donors', 'int32')]

# All fields that will be in the database
all_fields = main_fields + occurrence_global_fields + tuple_fields

# <-- Plain data structures
                             
# Single valued fields of a mutation
fnames = [fname for fname,ftype in main_fields + occurrence_global_fields]
MutationSimple = namedtuple('MutationSimple', fnames)
                             
# Complete data for a mutation
MutationComplete = namedtuple('MutationComplete', ['simple', 
                                                   'consequences', 
                                                   'occurrence_by_project'])

In [4]:
# < --- VCF file parsing

def clean_mutations(filename):
    """
    Helper function that returns a generator of the mutations parsed from the file.
    
    The format is suitable for creating a first file for database setup.
    """
    for raw_mutation in open_vcf(filename):
        # Yield the parsed mutation
        yield clean_mutation(raw_mutation)
# ---

def open_vcf(filename):
    """A generator to handle VCF files easily.
    
    Open the file with the given filename and yield every line that doesn't
    start with '#' (such as comment and header lines)
    """
    # Open the VCF file
    with open(filename) as file:
        # Yield only the lines that aren't comments
        for line in file:
            if not line.startswith('#'):
                yield line
# ---

# < --- Mutation parsing

# < -- Parse the raw mutation 

def clean_mutation(raw_mutation):
    'Decompose a raw mutation line into fields, returns it as a dict'
    # Split the mutation into fields and eliminate newline
    *main_fields, rawINFO = raw_mutation.strip().split('\t')
    # Parse the INFO
    (consequences, 
     occurrence_by_project, 
     occurrence_global) = clean_INFO(rawINFO)
    # Assemble the mutation
    mutation = MutationSimple(*main_fields, 
                              *occurrence_global)
    # Return the mutation
    return MutationComplete(mutation, consequences, occurrence_by_project)
# ---

def clean_INFO(rawINFO):
    """Parse the raw text INFO field of a mutation into:
        - consecuences
        - occurrence_by_project
        - occurrence_global
    """
    # Split the data into fields
    (consequences, 
     occurrence_by_project, 
     *occurrence_global) = rawINFO.split(';')
    # Clean the fields
    consequences = clean_tuple_field(consequences)
    occurrence_by_project = clean_tuple_field(occurrence_by_project)
    occurrence_global = clean_occurrence_global(occurrence_global)
    
    return consequences, occurrence_by_project, occurrence_global
# ---

def clean_tuple_field(raw_field):
    """Cleanup of a field that may have a variable number of values.
    
    A tuple field is of the form 'FIELD_NAME=value1,value2,...'
    
    Returns (value1, value2, ...)
    """
    # Remove the trailing 'FIELD_NAME=' string
    _ , trimmed_field = raw_field.split('=')
    # Separate each key, value pair
    values = trimmed_field.split(',')
    
    return values
# ---

def clean_occurrence_global(raw_occurrence):
    """Split the raw comma-separated fields of the global occurrence.
        
    Accepts a collection of elements of the form 'KEY=value'.
    Returns a colection with the values 'value'
    """
    # Separate into the corresponding fields
    occurrence = ( keyvalue.split('=') for keyvalue in raw_occurrence )
    return [value for key,value in occurrence]

In [5]:
def mutations_to_csv(mutations, separator=',', header=False):
    """Return a table representation of the mutations file.
    
    Creates a generator that dispatches line by line.
    """
    # Header (schema) line
    if header:
        yield separator.join([fname for fname,ftype in all_fields])

    # From the complete data, assemble the database items
    for mutation in mutations:
        # Begin assembling the output
        mutation_prefix = separator.join(mutation.simple)
        # Each line must have only one of the items of
        # the multiple values in the occurrence_by_project
        # and consequence part of the data.
        for occurrence in mutation.occurrence_by_project:
            for consequence in mutation.consequences:
                # Mutation line
                yield separator.join([mutation_prefix, 
                                      consequence,
                                      occurrence])

In [6]:
# Open the file and read the mutations
mutations = mutations_to_csv(
                clean_mutations('../data/ssm_mixed.vcf'))

In [12]:
# Open the file and read the mutations
muts = clean_mutations('../data/ssm_mixed.vcf')
m = next(muts)
pprint(m.consequences)

['||||||intergenic_region||',
 'S1PR1|ENSG00000170989|+|S1PR1-001|ENST00000305352||upstream_gene_variant||',
 'RP4-575N6.4|ENSG00000225938|1|RP4-575N6.4-001|ENST00000432195||downstream_gene_variant||',
 'snoU13|ENSG00000238296|1|snoU13.3-201|ENST00000459166||upstream_gene_variant||']


In [13]:
%time

# We create a header-less file due to the ODO library

with open('../data/ssm_mixed_nohead.csv', 'w') as file:
    for mutation in mutations:
        file.write(mutation + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


In [14]:
# Assemble the datashape of the data
partial_dshape = ",\n".join(key + ':' + value 
                            for key, value in all_fields) 

datashape = 'var * {\n' + partial_dshape + '\n}'
print(datashape)

var * {
chromosome:string,
GRCh37_pos:int32,
mutation_id:string,
reference_allele:string,
mutated_allele:string,
quality:string,
filter:string,
affected_donors:int32,
mutation:string,
project_count:int32,
tested_donors:int32,
consequence:string,
occurrence_by_project:string
}


In [32]:
import odo # Database loader

In [41]:
# Load to the database
dbfile = '../data/ssm_mixed2.sqlite'

database = odo.resource(f'sqlite:///{dbfile}::ssm',
         dshape=datashape,
         primary_key=['mutation_id'])

odo.odo('../data/ssm_mixed_nohead.csv', database)

Table('ssm', MetaData(bind=Engine(sqlite:///../data/ssm_mixed2.sqlite)), Column('chromosome', Text(), table=<ssm>, nullable=False), Column('GRCh37_pos', Integer(), table=<ssm>, nullable=False), Column('mutation_id', Text(), table=<ssm>, nullable=False), Column('reference_allele', Text(), table=<ssm>, nullable=False), Column('mutated_allele', Text(), table=<ssm>, nullable=False), Column('quality', Text(), table=<ssm>, nullable=False), Column('filter', Text(), table=<ssm>, nullable=False), Column('affected_donors', Integer(), table=<ssm>, nullable=False), Column('mutation', Text(), table=<ssm>, nullable=False), Column('project_count', Integer(), table=<ssm>, nullable=False), Column('tested_donors', Integer(), table=<ssm>, nullable=False), Column('consequence', Text(), table=<ssm>, nullable=False), Column('occurrence_by_project', Text(), table=<ssm>, nullable=False), schema=None)

In [42]:
import records

db = records.Database(f'sqlite:///{dbfile}')
rows = db.query('select * from ssm')

In [43]:
print(rows[:10].dataset)

chromosome|GRCh37_pos|mutation_id|reference_allele|mutated_allele|quality|filter|affected_donors|mutation|project_count|tested_donors|consequence                                                                             |occurrence_by_project
----------|----------|-----------|----------------|--------------|-------|------|---------------|--------|-------------|-------------|----------------------------------------------------------------------------------------|---------------------
1         |101697478 |MU56006960 |C               |G             |.      |.     |1              |C>G     |1            |10648        |||||||intergenic_region||                                                               |LUSC-KR|1|66|0.01515 
1         |101697478 |MU56006960 |C               |G             |.      |.     |1              |C>G     |1            |10648        |S1PR1|ENSG00000170989|+|S1PR1-001|ENST00000305352||upstream_gene_variant||              |LUSC-KR|1|66|0.01515 
1         |101697478

In [184]:
rows[0]

<Record {"chromosome": "1", "GRCh37_pos": 101697478, "mutation_id": "MU56006960", "reference_allele": "C", "mutated_allele": "G", "quality": ".", "filter": ".", "consequence": "1", "occurrence_by_project": "C>G", "affected_donors": 1, "mutation": "10648", "project_count": "LUSC-KR|1|66|0.01515", "tested_donors": "||||||intergenic_region||"}>

In [185]:
help(rows[0])

Help on Record in module records object:

class Record(builtins.object)
 |  A row, from a query, from a database.
 |  
 |  Methods defined here:
 |  
 |  __dir__(self)
 |      __dir__() -> list
 |      default dir() implementation
 |  
 |  __getattr__(self, key)
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, keys, values)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  as_dict(self, ordered=False)
 |      Returns the row as a dictionary, as ordered.
 |  
 |  export(self, format, **kwargs)
 |      Exports the row to the given format.
 |  
 |  get(self, key, default=None)
 |      Returns the value for a given key, or default.
 |  
 |  keys(self)
 |      Returns the list of column names from the query.
 |  
 |  values(self)
 |      Returns the list of values from the query.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  da

In [19]:
chr1 = db.query('select mutation_id,occurrence_by_project from ssm')

In [20]:
print(chr1[:10].dataset)

mutation_id|occurrence_by_project
-----------|---------------------
MU56006960 |LUSC-KR|1|66|0.01515 
MU56006960 |LUSC-KR|1|66|0.01515 
MU56006960 |LUSC-KR|1|66|0.01515 
MU56006960 |LUSC-KR|1|66|0.01515 
MU66170198 |BRCA-EU|1|560|0.00179
MU66170198 |BRCA-EU|1|560|0.00179
MU66170198 |BRCA-EU|1|560|0.00179
MU66170198 |BRCA-EU|1|560|0.00179
MU66170198 |BRCA-EU|1|560|0.00179
MU28940329 |PACA-CA|1|259|0.00386


In [21]:
chr1 = db.query('select mutation_id,consequence from ssm')
print(chr1[:10].dataset)

mutation_id|consequence                                                                             
-----------|----------------------------------------------------------------------------------------
MU56006960 |||||||intergenic_region||                                                               
MU56006960 |S1PR1|ENSG00000170989|+|S1PR1-001|ENST00000305352||upstream_gene_variant||              
MU56006960 |RP4-575N6.4|ENSG00000225938|1|RP4-575N6.4-001|ENST00000432195||downstream_gene_variant||
MU56006960 |snoU13|ENSG00000238296|1|snoU13.3-201|ENST00000459166||upstream_gene_variant||          
MU66170198 |||||||intergenic_region||                                                               
MU66170198 |S1PR1|ENSG00000170989|+|S1PR1-001|ENST00000305352||upstream_gene_variant||              
MU66170198 |RP4-575N6.4|ENSG00000225938|1|RP4-575N6.4-001|ENST00000432195||downstream_gene_variant||
MU66170198 |snoU13|ENSG00000238296|1|snoU13.3-201|ENST00000459166||upstream_gene_variant|| 

In [31]:
conseq_q = db.query('select count(distinct consequence) from ssm')
print(conseq_q.dataset)

count(distinct consequence)
---------------------------
11743                      


In [34]:
q_q = db.query('select distinct filter from ssm')
print(q_q.dataset)

filter
------
.     


In [38]:
i

8

In [37]:
for i in range(9):
    print(i)

0
1
2
3
4
5
6
7
8
